# 🥔 Phase 2: Potato Disease Dataset Merging

This notebook merges multiple potato disease datasets from Google Drive into a single unified dataset.

**Features:**
- Mounts Google Drive and reads all dataset folders
- Identifies common disease classes across datasets
- Creates unified `merged_potato_dataset` directory
- Copies images with unique prefixes to avoid conflicts
- Generates merge statistics report

## 1. Mount Google Drive

Run the cell below to mount Google Drive. If you encounter errors, the code will automatically try to fix them.

In [ ]:
from google.colab import drive
import os
import shutil

MOUNT_PATH = '/content/drive'

def mount_drive():
    """Mount Google Drive with automatic error handling."""
    
    # Check if already mounted and working
    if os.path.exists(os.path.join(MOUNT_PATH, 'MyDrive')):
        print('✅ Google Drive is already mounted!')
        return True
    
    # Clear the mount point if it exists and has files
    if os.path.exists(MOUNT_PATH):
        print('🔄 Clearing existing mount point...')
        try:
            # Try to unmount first
            drive.flush_and_unmount()
            print('   Unmounted previous session.')
        except:
            pass
        
        # Remove the directory if it still exists
        if os.path.exists(MOUNT_PATH):
            try:
                shutil.rmtree(MOUNT_PATH)
                print('   Cleared mount directory.')
            except Exception as e:
                print(f'   Warning: Could not clear directory: {e}')
    
    # Now mount
    try:
        drive.mount(MOUNT_PATH)
        print('✅ Google Drive mounted successfully!')
        return True
    except Exception as e:
        print(f'❌ Mount failed: {e}')
        print('\n🔧 MANUAL FIX:')
        print('   1. Go to Runtime → Restart runtime')
        print('   2. Run this cell again')
        return False

# Mount drive
mount_drive()

In [ ]:
# Verify Drive is mounted and check your folder structure
drive_path = '/content/drive/MyDrive'

if os.path.exists(drive_path):
    print('✅ Drive mounted at:', drive_path)
    print('\n📁 Contents of MyDrive:')
    for item in sorted(os.listdir(drive_path))[:15]:
        print(f'   - {item}')
else:
    print('❌ Drive not mounted. Please run the mount cell above first.')

## 2. Configuration

Update these paths according to your Google Drive structure.

In [ ]:
import os
import shutil
from pathlib import Path
from collections import defaultdict
from datetime import datetime
import json

# ===== CONFIGURATION =====
# Path to the folder containing all potato dataset folders
DATASETS_ROOT = "/content/drive/MyDrive/DrukFarm/data/potatoes"

# Path where the merged dataset will be created
OUTPUT_DIR = "/content/drive/MyDrive/DrukFarm/data/merged_potato_dataset"

# Valid image extensions
VALID_IMAGE_EXTENSIONS = {'.jpg', '.jpeg', '.png', '.bmp', '.gif', '.webp', '.tiff', '.tif'}

# Class name normalization mapping (add more as needed)
# Maps various naming conventions to standardized names
CLASS_NAME_MAPPING = {
    # Early Blight variations
    'early_blight': 'Early_Blight',
    'early blight': 'Early_Blight',
    'earlyblight': 'Early_Blight',
    'early-blight': 'Early_Blight',
    'potato___early_blight': 'Early_Blight',
    'potato_early_blight': 'Early_Blight',
    
    # Late Blight variations
    'late_blight': 'Late_Blight',
    'late blight': 'Late_Blight',
    'lateblight': 'Late_Blight',
    'late-blight': 'Late_Blight',
    'potato___late_blight': 'Late_Blight',
    'potato_late_blight': 'Late_Blight',
    
    # Healthy variations
    'healthy': 'Healthy',
    'potato___healthy': 'Healthy',
    'potato_healthy': 'Healthy',
}

print("✅ Configuration loaded successfully!")
print(f"📁 Source datasets: {DATASETS_ROOT}")
print(f"📂 Output directory: {OUTPUT_DIR}")

# Check if source path exists
if os.path.exists(DATASETS_ROOT):
    print(f"\n✅ Source path found!")
    print(f"📁 Contents of {os.path.basename(DATASETS_ROOT)}:")
    for item in os.listdir(DATASETS_ROOT):
        item_path = os.path.join(DATASETS_ROOT, item)
        if os.path.isdir(item_path):
            print(f"   📁 {item}/")
        else:
            print(f"   📄 {item}")
else:
    print(f"\n❌ Source path NOT found: {DATASETS_ROOT}")
    print("Please update DATASETS_ROOT to match your Drive folder structure.")

## 3. Utility Functions

In [ ]:
def normalize_class_name(class_name: str) -> str:
    """
    Normalize class name to a standard format.
    Handles various naming conventions from different datasets.
    """
    # Convert to lowercase for mapping lookup
    lower_name = class_name.lower().strip()
    
    # Check if we have a predefined mapping
    if lower_name in CLASS_NAME_MAPPING:
        return CLASS_NAME_MAPPING[lower_name]
    
    # If not in mapping, apply general normalization
    # Replace spaces and hyphens with underscores, capitalize each word
    normalized = class_name.replace('-', '_').replace(' ', '_')
    normalized = '_'.join(word.capitalize() for word in normalized.split('_'))
    
    return normalized


def is_valid_image(file_path: str) -> bool:
    """
    Check if a file is a valid image based on its extension.
    """
    return Path(file_path).suffix.lower() in VALID_IMAGE_EXTENSIONS


def get_unique_filename(dest_dir: str, original_name: str, dataset_prefix: str) -> str:
    """
    Generate a unique filename to avoid conflicts.
    Format: {dataset_prefix}_{original_name}
    If still conflicts, adds numeric suffix.
    """
    stem = Path(original_name).stem
    suffix = Path(original_name).suffix
    
    # Create new filename with dataset prefix
    new_name = f"{dataset_prefix}_{stem}{suffix}"
    new_path = os.path.join(dest_dir, new_name)
    
    # If file still exists, add numeric suffix
    counter = 1
    while os.path.exists(new_path):
        new_name = f"{dataset_prefix}_{stem}_{counter}{suffix}"
        new_path = os.path.join(dest_dir, new_name)
        counter += 1
    
    return new_name


def sanitize_prefix(name: str) -> str:
    """
    Sanitize dataset name to create a valid file prefix.
    """
    # Replace invalid characters with underscore
    sanitized = ''.join(c if c.isalnum() or c in '-_' else '_' for c in name)
    return sanitized[:20]  # Limit length


print("✅ Utility functions defined!")

## 4. Dataset Discovery

In [ ]:
def discover_datasets(root_path: str) -> dict:
    """
    Discover all datasets and their classes in the root directory.
    Returns a dictionary mapping dataset names to their class folders.
    """
    datasets = {}
    
    if not os.path.exists(root_path):
        print(f"❌ Error: Root path does not exist: {root_path}")
        return datasets
    
    # Iterate through items in the root directory
    for item in os.listdir(root_path):
        item_path = os.path.join(root_path, item)
        
        # Skip if not a directory
        if not os.path.isdir(item_path):
            print(f"⏭️ Skipping non-directory: {item}")
            continue
        
        # Check if this directory contains class subdirectories
        classes = {}
        for class_item in os.listdir(item_path):
            class_path = os.path.join(item_path, class_item)
            
            if os.path.isdir(class_path):
                # Count images in this class directory
                image_count = sum(
                    1 for f in os.listdir(class_path)
                    if is_valid_image(os.path.join(class_path, f))
                )
                if image_count > 0:
                    normalized_class = normalize_class_name(class_item)
                    classes[class_item] = {
                        'path': class_path,
                        'normalized_name': normalized_class,
                        'image_count': image_count
                    }
        
        if classes:
            datasets[item] = classes
            print(f"📁 Found dataset: {item} with {len(classes)} classes")
        else:
            print(f"⚠️ Skipping folder (no valid classes): {item}")
    
    return datasets


# Discover datasets
print("🔍 Discovering datasets...\n")
discovered_datasets = discover_datasets(DATASETS_ROOT)

print(f"\n✅ Discovered {len(discovered_datasets)} dataset(s)")

## 5. Analyze Classes Across Datasets

In [ ]:
def analyze_classes(datasets: dict) -> dict:
    """
    Analyze and aggregate class information across all datasets.
    """
    class_summary = defaultdict(lambda: {'datasets': [], 'total_images': 0})
    
    for dataset_name, classes in datasets.items():
        for original_class, info in classes.items():
            normalized = info['normalized_name']
            class_summary[normalized]['datasets'].append({
                'dataset': dataset_name,
                'original_name': original_class,
                'path': info['path'],
                'image_count': info['image_count']
            })
            class_summary[normalized]['total_images'] += info['image_count']
    
    return dict(class_summary)


# Analyze classes
class_analysis = analyze_classes(discovered_datasets)

print("📊 CLASS ANALYSIS SUMMARY")
print("=" * 50)
for class_name, info in sorted(class_analysis.items()):
    print(f"\n🏷️ {class_name}:")
    print(f"   Total images: {info['total_images']}")
    print(f"   Found in {len(info['datasets'])} dataset(s):")
    for ds in info['datasets']:
        print(f"      - {ds['dataset']}: {ds['image_count']} images (as '{ds['original_name']}')") 

## 6. Merge Datasets

In [ ]:
def merge_datasets(class_analysis: dict, output_dir: str) -> dict:
    """
    Merge all datasets into a unified directory structure.
    Returns statistics about the merge operation.
    """
    stats = {
        'classes': {},
        'total_images_copied': 0,
        'skipped_files': 0,
        'errors': []
    }
    
    # Create output directory
    os.makedirs(output_dir, exist_ok=True)
    print(f"📂 Created output directory: {output_dir}\n")
    
    # Process each normalized class
    for class_name, info in sorted(class_analysis.items()):
        print(f"\n🔄 Processing class: {class_name}")
        
        # Create class directory
        class_dir = os.path.join(output_dir, class_name)
        os.makedirs(class_dir, exist_ok=True)
        
        class_stats = {
            'images_copied': 0,
            'sources': {}
        }
        
        # Copy images from each dataset
        for dataset_info in info['datasets']:
            dataset_name = dataset_info['dataset']
            source_path = dataset_info['path']
            prefix = sanitize_prefix(dataset_name)
            
            copied_count = 0
            
            # Copy each image
            for filename in os.listdir(source_path):
                source_file = os.path.join(source_path, filename)
                
                # Skip non-image files
                if not is_valid_image(source_file):
                    stats['skipped_files'] += 1
                    continue
                
                # Skip directories
                if os.path.isdir(source_file):
                    continue
                
                try:
                    # Generate unique filename
                    new_filename = get_unique_filename(class_dir, filename, prefix)
                    dest_file = os.path.join(class_dir, new_filename)
                    
                    # Copy file
                    shutil.copy2(source_file, dest_file)
                    copied_count += 1
                    stats['total_images_copied'] += 1
                    
                except Exception as e:
                    error_msg = f"Error copying {source_file}: {str(e)}"
                    stats['errors'].append(error_msg)
                    print(f"   ❌ {error_msg}")
            
            class_stats['sources'][dataset_name] = copied_count
            class_stats['images_copied'] += copied_count
            print(f"   ✅ Copied {copied_count} images from '{dataset_name}'")
        
        stats['classes'][class_name] = class_stats
        print(f"   📊 Total for {class_name}: {class_stats['images_copied']} images")
    
    return stats


# Execute merge
print("🚀 STARTING DATASET MERGE")
print("=" * 50)

merge_stats = merge_datasets(class_analysis, OUTPUT_DIR)

print("\n" + "=" * 50)
print("✅ MERGE COMPLETED!")

## 7. Generate Merge Report

In [ ]:
def generate_report(stats: dict, output_dir: str) -> None:
    """
    Generate and display a comprehensive merge report.
    Also saves the report to a JSON file.
    """
    print("\n" + "=" * 60)
    print("📋 MERGE REPORT")
    print("=" * 60)
    
    print(f"\n📅 Merge Date: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"📂 Output Directory: {output_dir}")
    print(f"\n📊 SUMMARY:")
    print(f"   • Total images copied: {stats['total_images_copied']}")
    print(f"   • Total classes created: {len(stats['classes'])}")
    print(f"   • Non-image files skipped: {stats['skipped_files']}")
    print(f"   • Errors encountered: {len(stats['errors'])}")
    
    print(f"\n📁 CLASS BREAKDOWN:")
    print("-" * 40)
    
    for class_name, class_stats in sorted(stats['classes'].items()):
        print(f"\n   🏷️ {class_name}: {class_stats['images_copied']} images")
        for source, count in class_stats['sources'].items():
            print(f"      └─ {source}: {count}")
    
    if stats['errors']:
        print(f"\n⚠️ ERRORS:")
        for error in stats['errors'][:10]:  # Show first 10 errors
            print(f"   • {error}")
        if len(stats['errors']) > 10:
            print(f"   ... and {len(stats['errors']) - 10} more errors")
    
    # Save report to JSON
    report_path = os.path.join(output_dir, "merge_report.json")
    report_data = {
        'merge_date': datetime.now().isoformat(),
        'output_directory': output_dir,
        'total_images_copied': stats['total_images_copied'],
        'total_classes': len(stats['classes']),
        'skipped_files': stats['skipped_files'],
        'errors_count': len(stats['errors']),
        'class_details': stats['classes'],
        'errors': stats['errors']
    }
    
    with open(report_path, 'w') as f:
        json.dump(report_data, f, indent=2)
    
    print(f"\n💾 Report saved to: {report_path}")


# Generate report
generate_report(merge_stats, OUTPUT_DIR)

## 8. Verify Merged Dataset

In [ ]:
def verify_merged_dataset(output_dir: str) -> None:
    """
    Verify the merged dataset structure and contents.
    """
    print("\n" + "=" * 60)
    print("🔍 VERIFICATION")
    print("=" * 60)
    
    if not os.path.exists(output_dir):
        print("❌ Output directory does not exist!")
        return
    
    print(f"\n📂 Merged Dataset Structure:")
    print(f"{os.path.basename(output_dir)}/")
    
    total_images = 0
    
    for class_folder in sorted(os.listdir(output_dir)):
        class_path = os.path.join(output_dir, class_folder)
        
        if os.path.isdir(class_path):
            image_count = len([
                f for f in os.listdir(class_path)
                if is_valid_image(os.path.join(class_path, f))
            ])
            total_images += image_count
            print(f"├── {class_folder}/  ({image_count} images)")
    
    print(f"\n✅ Total verified images: {total_images}")
    print("\n🎉 Dataset is ready for preprocessing and model training!")


# Verify
verify_merged_dataset(OUTPUT_DIR)

## 9. Sample Images Preview (Optional)

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image
import random

def preview_samples(output_dir: str, samples_per_class: int = 3) -> None:
    """
    Display sample images from each class in the merged dataset.
    """
    classes = sorted([
        d for d in os.listdir(output_dir)
        if os.path.isdir(os.path.join(output_dir, d))
    ])
    
    if not classes:
        print("No classes found!")
        return
    
    fig, axes = plt.subplots(len(classes), samples_per_class, figsize=(12, 4 * len(classes)))
    
    if len(classes) == 1:
        axes = [axes]
    
    for i, class_name in enumerate(classes):
        class_path = os.path.join(output_dir, class_name)
        images = [
            f for f in os.listdir(class_path)
            if is_valid_image(os.path.join(class_path, f))
        ]
        
        # Sample random images
        sample_images = random.sample(images, min(samples_per_class, len(images)))
        
        for j, img_name in enumerate(sample_images):
            img_path = os.path.join(class_path, img_name)
            img = Image.open(img_path)
            
            ax = axes[i][j] if len(classes) > 1 else axes[j]
            ax.imshow(img)
            ax.set_title(f"{class_name}\n{img_name[:30]}...", fontsize=8)
            ax.axis('off')
        
        # Fill remaining slots with blank
        for j in range(len(sample_images), samples_per_class):
            ax = axes[i][j] if len(classes) > 1 else axes[j]
            ax.axis('off')
    
    plt.tight_layout()
    plt.suptitle("Sample Images from Merged Dataset", fontsize=14, y=1.02)
    plt.show()


# Preview samples
preview_samples(OUTPUT_DIR)

---

## ✅ Phase 2 Complete!

The merged dataset has been created at:
```
/content/drive/MyDrive/DrukFarm/data/merged_potato_dataset/
├── Early_Blight/
├── Late_Blight/
└── Healthy/
```

**Next Steps:**
- Phase 3: Data Preprocessing (resizing, augmentation)
- Phase 4: Model Training